# Day3 — Univariate Forecast (Prophet) + MAE
목표:
- Prophet 단변량 예측을 먼저 적용해보고(figures/Day3)
- 실제 vs 예측 비교 그래프 생성
- MAE 계산 후 표 저장(tables/Day3/day3_mae.csv)
- (주의) split에 따라 test 구간이 정지(RPM=0)로 치우칠 수 있음 → 로그로 확인

## 입력/출력
- 입력(processed): data/processed/processed_v1_run_abs_features.csv
- 출력(표): tables/Day3/ (day3_mae.csv)
- 출력(그림): figures/Day3/ (forecast_*.png)
- 실행 스크립트: src/day3_forecast_univariate.py

In [ ]:
from pathlib import Path
import pandas as pd

PROCESSED = Path("../../data/processed/processed_v1_run_abs_features.csv")
TABLES_DIR = Path("../../tables/Day3")
FIG_DIR = Path("../../figures/Day3")

PROCESSED.exists(), TABLES_DIR.exists(), FIG_DIR.exists()

## 1) 데이터 로드 & 기본 확인 (분포/스케일 sanity check)

In [ ]:
df = pd.read_csv(PROCESSED)
df.shape, df.columns[:15]

## 2) Day3 산출물 생성 (스크립트 실행)
- Prophet 단변량 예측: Temp_mean, FB_Torque
- resample=1s
- run 기반 train/test split
- MAE 계산 및 저장

In [ ]:
import subprocess
import sys

def run(cmd: str):
    print(">", cmd)
    res = subprocess.run(cmd, shell=True, text=True, capture_output=True)
    print(res.stdout)
    if res.returncode != 0:
        print(res.stderr)
    return res.returncode

py = sys.executable
run(f"{py} src/day3_forecast_univariate.py")

## 3) 생성 결과 확인 (파일 리스트)

In [ ]:
TABLES_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

tables = sorted(TABLES_DIR.glob("*.csv"))
figs = sorted(FIG_DIR.glob("*.png"))

[t.name for t in tables], [f.name for f in figs]

## 4) MAE 결과 확인 (표 미리보기)

In [ ]:
mae_path = TABLES_DIR / "day3_mae.csv"
mae_df = pd.read_csv(mae_path)
mae_df

## 5) 예측 그래프 확인 (노트북에서 이미지 확인)
- forecast_Temp_mean.png
- forecast_FB_Torque.png

In [ ]:
from IPython.display import Image, display

for p in sorted(FIG_DIR.glob("forecast_*.png")):
    print(p.name)
    display(Image(filename=str(p)))

## 6) Day3 관찰 요약
- Temp_mean: 변화폭이 작아 MAE가 작게 나오는 경향
- FB_Torque: 구간 중 급격한 전환(레짐 변화)이 포함되면 단변량 예측이 평균값에 수렴해 MAE가 커질 수 있음
- run split 시 test가 정지(RPM=0)로 치우칠 수 있으므로, 실행 로그(running_ratio)를 함께 확인

## Day4 계획 (여러 센서로 예측)
- 토크 예측 시 RPM/진동/온도/압력 등 다른 센서를 함께 활용
- 단변량 vs 여러 센서 기반 예측 결과를 그래프/MAE로 비교